- https://github.com/teddylee777/langchain-kr/blob/main/17-LangGraph/02-langgraph-groundcheck.ipynb
- https://www.youtube.com/watch?v=4JdzuB702wI&t=1s

# LangGraph Retrieval GroundCheck
기존의 RAG에 GroundChecker 기능을 추가하여 답변-문서 간에 관련성 여부를 확인합니다.

이번 튜토리얼에서는 직접 구현한 LLM 으로 GroundChecker를 만들어서 관련성 여부를 체크 합니다.

# 환경설정

In [ ]:
# api key
from dotenv import load_dotenv

load_dotenv()

# LangSmith 설졍

In [ ]:
import os

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]="ls__708b8970829247d1a055f33c434aad1d"
os.environ["LANGCHAIN_PROJECT"]="edu-langchain-0326"

### VectorDB RAG 생성

In [ ]:
from rag.pdf import PDFRetrievalChain

# PDF 문서를 vector로 변환하여 retriever 생성
pdf = PDFRetrievalChain(["data/SPRI_AI_Brief_2023년12월호_F.pdf"]).create_chain()
pdf_retriever = pdf.retriever
pdf_chain = pdf.chain

# GraphState
각 노드에서 다음 노드로 전달되는 상태를 저장합니다.

In [ ]:
from typing import TypedDict


# GraphState 상태를 저장하는 용도로 사용합니다.
class GraphState(TypedDict):
    question: str  # 질문
    context: str  # 문서의 검색 결과
    answer: str  # 답변
    relevance: str  # 답변의 문서에 대한 관련성

# 노드와 엣지
주요 개념

- GraphState(상태 저장 그래프): LangGraph는 그래프의 각 노드가 계산의 단계를 나타내며, 그래프는 계산이 진행됨에 따라 전달되고 업데이트되는 상태를 유지하는 상태 저장 그래프 개념을 중심으로 작동합니다.
- Node(노드): 노드는 LangGraph의 구성 요소입니다. 각 노드는 함수 또는 계산 단계를 나타냅니다. 입력 처리, 의사 결정, 외부 API와의 상호 작용 등 특정 작업을 수행하도록 노드를 정의할 수 있습니다.
- Edge(엣지): 에지는 그래프에서 노드를 연결하여 계산의 흐름을 정의합니다. LangGraph는 조건부 에지를 지원하므로 그래프의 현재 상태에 따라 실행할 다음 노드를 동적으로 결정할 수 있습니다.

In [ ]:
from rag.utils import format_docs
from langchain_upstage import UpstageGroundednessCheck

os.environ["UPSTAGE_API_KEY"] = "up_gBj0DmykHNhYdGNmB4JLWT7iVlKVF"

# 업스테이지 문서 관련성 체크 기능을 설정합니다. https://upstage.ai
upstage_ground_checker = UpstageGroundednessCheck()

# 업스테이지 문서 관련성 체크를 실행합니다.
upstage_ground_checker.run(
    {
        "context": format_docs(
            pdf_retriever.invoke("삼성전자가 개발한 생성AI 의 이름은")
        ),
        "answer": "삼성전자가 개발한 생성AI 의 이름은 `빅스비 AI` 입니다.",
    }
)

LLM 객체 생성

In [ ]:
from langchain_openai import AzureChatOpenAI

def Get_LLM():    
    os.environ["AZURE_OPENAI_API_KEY"] = '352a6bee97b5451ab5866993a7ef4ce4'
    os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://aoai-spn-krc.openai.azure.com/'
    model = AzureChatOpenAI(  
      api_version = '2024-02-01',
      azure_deployment = 'gpt-4o-kr-spn',
      temperature = 0.0
    )
    return model

Chain 테스트

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 생성 - 다음 맥락을 기반으로만 질문에 답하십시오.
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = Get_LLM()

test_chain = (
    {"context": pdf_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# _question = "삼성에서 개발한 AI는?"
# _question = "가우스를 개발한 회사의 2023년 매출액은?"
# _question = "가우스는 언제 공개되었나?"
_question = "가우스를 개발한 회사의 이름과 매출액은?"

test_chain.invoke(_question)


상태 저장 클래스

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field

# 관련성 결과 형식
class GradeDocuments(BaseModel):
    """Boolean values to check for relevance on retrieved documents."""

    score: str = Field(
        description="Documents are relevant to the question, 'True' or 'False'"
    )

node 함수 생성

In [ ]:
from rag.utils import format_docs
from langchain_core.prompts import ChatPromptTemplate
from rag.base import RetrievalChain

# 벡터DB에서 문서를 검색합니다.
def retrieve_document(state: GraphState) -> GraphState:
    # Question 에 대한 문서 검색을 retriever 로 수행합니다.
    retrieved_docs = pdf_retriever.invoke(state["question"])
    # 검색된 문서를 context 키에 저장합니다.
    return GraphState(context=format_docs(retrieved_docs))


# Chain을 사용하여 답변을 생성합니다.
def llm_answer(state: GraphState) -> GraphState:
    return GraphState(
        answer=pdf_chain.invoke(
            {"question": state["question"], "context": state["context"]}
        ),
        context=state["context"],
        question=state["question"],
    )


# 테스트용 더미 답변 생성 함수
def fake_llm_answer(state: GraphState) -> GraphState:
    return GraphState(
        answer="삼성전자가 개발한 생성AI 의 이름은 `빅스비 AI` 입니다.",
        context=state["context"],
        question=state["question"],
    )


# 관련성 체크를 함수
def relevance_check(state: GraphState) -> GraphState:
    doc = state["context"]
    question = state["question"]

    # 시스템 프롬프트:
    # 당신은 사용자의 질문과 관련된 문서의 중요성을 평가하는 평가자입니다. \n
    # 문서가 질문과 관련된 키워드나 의미를 포함하고 있다면, 해당 문서를 관련성이 있는 것으로 평가하세요. \n
    # 문서가 질문과 관련이 있는지 여부를 나타내기 위해 'True' 또는 'False'라는 불린 점수를 부여하세요.

    system = """You are a grader assessing relevance of a retrieved document to a user question. \n
        If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant. \n
        Give a boolean score 'True' or 'False' score to indicate whether the document is relevant to the question."""

    grade_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            (
                "human",
                "Retrieved document: \n\n {document} \n\n User question: {question}",
            ),
        ]
    )

    llm = Get_LLM()
    structured_llm = llm.with_structured_output(GradeDocuments)
    grader_llm = grade_prompt | structured_llm
    result = grader_llm.invoke({"document": doc, "question": question})
    return GraphState(
        relevance=result.score,
        context=state["context"],
        answer=state["answer"],
        question=state["question"],
    )

# 관련성 체크 결과를 반환합니다.
def is_relevant(state: GraphState) -> GraphState:
    if state["relevance"].lower() == "true":
        return "관련성 O"
    else:
        return "관련성 X"

# 관련성 체크를 실행합니다.
# def relevance_check(state: GraphState) -> GraphState:
#     # 관련성 체크를 실행합니다. 결과: grounded, notGrounded, notSure
#     response = upstage_ground_checker.run(
#         {"context": state["context"], "answer": state["answer"]}
#     )
#     return GraphState(
#         relevance=response,
#         context=state["context"],
#         answer=state["answer"],
#         question=state["question"],
#     )


# 관련성 체크 결과를 반환합니다.
# def is_relevant(state: GraphState) -> GraphState:
#     if state["relevance"] == "grounded":
#         return "관련성 O"
#     elif state["relevance"] == "notGrounded":
#         return "관련성 X"
#     elif state["relevance"] == "notSure":
#         return "확인불가"

# 그래프 정의

In [ ]:
from langgraph.graph import END, StateGraph
from langgraph.checkpoint.memory import MemorySaver

# langgraph.graph에서 StateGraph와 END를 가져옵니다.
workflow = StateGraph(GraphState)

# 노드들을 정의합니다.
workflow.add_node("retrieve", retrieve_document)  # 에이전트 노드를 추가합니다.
workflow.add_node("llm_answer", llm_answer)  # 정보 검색 노드를 추가합니다.

# 테스트용 fake llm 을 추가합니다.
# workflow.add_node("llm_answer", fake_llm_answer)  # 정보 검색 노드를 추가합니다.


workflow.add_node(
    "relevance_check", relevance_check
)  # 답변의 문서에 대한 관련성 체크 노드를 추가합니다.

# 각 노드들을 연결합니다.
workflow.add_edge("retrieve", "llm_answer")  # 검색 -> 답변
workflow.add_edge("llm_answer", "relevance_check")  # 답변 -> 관련성 체크

# 조건부 엣지를 추가합니다.
workflow.add_conditional_edges(
    "relevance_check",  # 관련성 체크 노드에서 나온 결과를 is_relevant 함수에 전달합니다.
    is_relevant,
    {
        "관련성 O": END,  # 관련성이 있으면 종료합니다.
        "관련성 X": "retrieve",  # 관련성이 없으면 다시 답변을 생성합니다.
    },
)

# 시작점을 설정합니다.
workflow.set_entry_point("retrieve")

# 기록을 위한 메모리 저장소를 설정합니다.
memory = MemorySaver()

# 그래프를 컴파일합니다.
app = workflow.compile(checkpointer=memory)

# 그래프 시각화

In [ ]:
from IPython.display import Image, display

try:
    display(
        Image(app.get_graph(xray=True).draw_mermaid_png())
    )  # 실행 가능한 객체의 그래프를 mermaid 형식의 PNG로 그려서 표시합니다. xray=True는 추가적인 세부 정보를 포함합니다.
except:
    # 이 부분은 추가적인 의존성이 필요하며 선택적으로 실행됩니다.
    pass

In [ ]:
from langgraph.pregel import GraphRecursionError

def execute_graph(app, inputs, config):
    try:
        result = app.invoke(inputs, config)
        return result
    except GraphRecursionError as e:
        print(f"Encountered a recursion error: {e}")
        # Custom error handling logic here
        return None

In [ ]:
from langchain_core.runnables import RunnableConfig

# recursion_limit: 최대 반복 횟수, thread_id: 실행 ID (구분용)
config = RunnableConfig(recursion_limit=13, configurable={"thread_id": "SELF-RAG"})

_question = "삼성에서 개발한 LLM"
# _question = "AI 가우스를 만든 회사 매출액"

# GraphState 객체를 활용하여 질문을 입력합니다.
inputs = GraphState(question=_question)
# output = app.invoke(inputs, config=config)
output = execute_graph(app, inputs, config)

if output is not None:
    # 출력 결과를 확인합니다.
    print("Question: \t", output["question"])
    print("Answer: \t", output["answer"])
    print("Relevance: \t", output["relevance"])

In [ ]:
import pprint
from langgraph.errors import GraphRecursionError
from langchain_core.runnables import RunnableConfig

config = RunnableConfig(recursion_limit=13, configurable={"thread_id": "SELF-RAG"})

_question = "삼성에서 개발한 LLM"
# _question = "AI 가우스를 만든 회사 매출액"

# GraphState 객체를 활용하여 질문을 입력합니다.
inputs = GraphState(question=_question)

# app.stream을 통해 입력된 메시지에 대한 출력을 스트리밍합니다.
try:
    for output in app.stream(inputs, config=config):
        # 출력된 결과에서 키와 값을 순회합니다.
        for key, value in output.items():
            # 노드의 이름과 해당 노드에서 나온 출력을 출력합니다.
            pprint.pprint(f"[NODE] {key}")
            for k, v in value.items():
                pprint.pprint(f"<{k}> {v}")
            pprint.pprint("===" * 10)
            # 출력 값을 예쁘게 출력합니다.
            # pprint.pprint(value, indent=2, width=80, depth=None)
except GraphRecursionError as e:
    pprint.pprint(f"Recursion limit reached: {e}")